# Code Demonstrating Functionality of Acquiring Data
Excerpts from demo.ipynb selected to clearly outline functionality of acquiring data

## Some Notes about get_acoustic_data
- There is a function get_acoustic_data_mp which uses multiprocessing. This function calls get_acoustic_data (which does not use multiprocessing.
- no filter is used for this example

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import datetime
import ooi_data_processing_library as ooipy
import timeit


# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
from dask import compute

In [3]:
# START DASK
from dask.distributed import Client
from dask import delayed

client = Client()

In [4]:
# 4 workers, 2 threads each = 8 threads
client

Client Scheduler: tcp://127.0.0.1:37903 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.52 GB


In [10]:
%%time
# get data using get_acoustic_data function
# starttime, endtime, and node have to be specified.
start_time = datetime.datetime(2016,10,6,0,0,0) # time of first sample
end_time = datetime.datetime(2016,10,6,0,5,0) # time of last sample
ooi1 = ooipy.OOIHydrophoneData()
ooi1.get_acoustic_data_conc(starttime=start_time, endtime=end_time, node='/PC01A', fmin=None, fmax=None, max_workers=8)
print('type: ', type(ooi1.data), '\n')
print('acoustic data: ', ooi1.data, '\n')
print('stats: ', ooi1.data.stats, '\n')
print('data: ', ooi1.data.data, '\n')

type:  <class 'obspy.core.trace.Trace'> 

acoustic data:  OO.HYVM2..YDH | 2016-10-06T00:00:00.000000Z - 2016-10-06T00:05:00.000000Z | 64000.0 Hz, 19200001 samples 

stats:           network: OO
         station: HYVM2
        location: 
         channel: YDH
       starttime: 2016-10-06T00:00:00.000000Z
         endtime: 2016-10-06T00:05:00.000000Z
   sampling_rate: 64000.0
           delta: 1.5625e-05
            npts: 19200001
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'D', 'number_of_records': 6816, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 4096, 'filesize': 27918336})
      processing: ['ObsPy 1.2.2: trim(endtime=UTCDateTime(2016, 10, 6, 0, 5)::fill_value=None::nearest_sample=True::pad=False::starttime=UTCDateTime(2016, 10, 6, 0, 0))'] 

data:  [-23994.         -23951.         -23779.         ..., -23547.74121406
 -23547.37060703 -23547.        ] 

CPU times: user 6.68 s, sys: 2.54 s, total: 9.22 s
Wall time: 18.3 s


In [6]:
%%time
# ==== WITH DASK ====
# get data using get_acoustic_data function
# starttime, endtime, and node have to be specified.
start_time = datetime.datetime(2016,10,6,0,0,0) # time of first sample
end_time = datetime.datetime(2016,10,6,0,5,0) # time of last sample
ooi1 = ooipy.OOIHydrophoneData()
ooi1.get_acoustic_data_conc(starttime=start_time, endtime=end_time, node='/PC01A', fmin=None, fmax=None, dask_client=client)
print('type: ', type(ooi1.data), '\n')
print('acoustic data: ', ooi1.data, '\n')
print('stats: ', ooi1.data.stats, '\n')
print('data: ', ooi1.data.data, '\n')

type:  <class 'obspy.core.trace.Trace'> 

acoustic data:  OO.HYVM2..YDH | 2016-10-06T00:00:00.000000Z - 2016-10-06T00:05:00.000000Z | 64000.0 Hz, 19200001 samples 

stats:           network: OO
         station: HYVM2
        location: 
         channel: YDH
       starttime: 2016-10-06T00:00:00.000000Z
         endtime: 2016-10-06T00:05:00.000000Z
   sampling_rate: 64000.0
           delta: 1.5625e-05
            npts: 19200001
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'D', 'number_of_records': 6816, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 4096, 'filesize': 27918336})
      processing: ['ObsPy 1.2.2: trim(endtime=UTCDateTime(2016, 10, 6, 0, 5)::fill_value=None::nearest_sample=True::pad=False::starttime=UTCDateTime(2016, 10, 6, 0, 0))'] 

data:  [-23994.         -23951.         -23779.         ..., -23547.74121406
 -23547.37060703 -23547.        ] 

CPU times: user 1.91 s, sys: 945 ms, total: 2.85 s
Wall time: 15 s


## Comparing Download Times for Different Nodes

In [9]:
%%time
start_time = datetime.datetime(2017,3,10,0,0,0) # time of first sample
end_time = datetime.datetime(2017,3,10,0,5,0) # time of last sample
ooi1 = ooipy.OOIHydrophoneData()
node1 = '/LJ01C'
print('Downloading Data from Node 1:')

t1_start = timeit.timeit()
ooi1.get_acoustic_data_conc(starttime=start_time, endtime=end_time, node=node1, fmin=None, fmax=None, verbose=True, max_workers=8)
t1_end = timeit.timeit()

ooi2 = ooipy.OOIHydrophoneData()
node2 = '/PC01A'

print('Downloading Data from Node 2:')
t2_start = timeit.timeit()
ooi2.get_acoustic_data_conc(starttime=start_time, endtime=end_time, node=node2, fmin=None, fmax=None, verbose=True, max_workers=8)
t2_end = timeit.timeit()

t1 = np.abs(t1_start-t1_end)
t2 = np.abs(t2_start-t2_end)
percent_increase = t2/t1*100

print('\n')
print(f'Time to Download 5 min from {node1}: {t1}')
print(f'Time to Download 5 min from {node2}: {t2}')
print('\n')
print(f'Percent Increase for Node 2: {percent_increase}%')

Fetching URLs...
Sorting valid URLs for Time Window...
Fetching URLs...
Sorting valid URLs for Time Window...


Time to Download 5 min from /LJ01C: 0.00015418899965879973
Time to Download 5 min from /PC01A: 0.0005312640005286084


Percent Increase for Node 2: 344.55376304679766%
CPU times: user 15.8 s, sys: 6.03 s, total: 21.8 s
Wall time: 37.6 s


In [7]:
%%time
# === WITH DASK ===
start_time = datetime.datetime(2017,3,10,0,0,0) # time of first sample
end_time = datetime.datetime(2017,3,10,0,5,0) # time of last sample
ooi1 = ooipy.OOIHydrophoneData()
node1 = '/LJ01C'
print('Downloading Data from Node 1:')

t1_start = timeit.timeit()
ooi1.get_acoustic_data_conc(starttime=start_time, endtime=end_time, node=node1, fmin=None, fmax=None, verbose=True, dask_client=client)
t1_end = timeit.timeit()

ooi2 = ooipy.OOIHydrophoneData()
node2 = '/PC01A'

print('Downloading Data from Node 2:')
t2_start = timeit.timeit()
ooi2.get_acoustic_data_conc(starttime=start_time, endtime=end_time, node=node2, fmin=None, fmax=None, verbose=True, dask_client=client)
t2_end = timeit.timeit()

t1 = np.abs(t1_start-t1_end)
t2 = np.abs(t2_start-t2_end)
percent_increase = t2/t1*100

print('\n')
print(f'Time to Download 5 min from {node1}: {t1}')
print(f'Time to Download 5 min from {node2}: {t2}')
print('\n')
print(f'Percent Increase for Node 2: {percent_increase}%')

Fetching URLs...
Sorting valid URLs for Time Window...
Fetching URLs...
Sorting valid URLs for Time Window...


Time to Download 5 min from /LJ01C: 0.0019345650007380755
Time to Download 5 min from /PC01A: 0.005332935000296857


Percent Increase for Node 2: 275.66584727120744%
CPU times: user 3.82 s, sys: 1.96 s, total: 5.78 s
Wall time: 30.3 s
